In [1]:
from IPython.display import clear_output
!pip install mediapipe
!pip install bing-image-downloader
!pip install deepface
clear_output()

In [2]:
import os
import json
import time
import cv2
import numpy as np
import mediapipe as mp
import math
import csv 

In [3]:
#@title utils

import cv2
import numpy as np
from deepface import DeepFace
import math
import time
def deep_data_extract(img):
    embedding=None
    faces=[]
    facial_data=[]
    try:
        embedding = DeepFace.represent(img, model_name='Facenet',detector_backend='ssd')
        if embedding:
          for  i in range(len(embedding)):
            x, y, w, h = embedding[i]['facial_area']['x'], embedding[i]['facial_area']['y'], embedding[i]['facial_area']['w'], embedding[i]['facial_area']['h']
            x1, y1, x2, y2 = x, y, x+w, y+h
            faces.append((x1, y1, x2, y2 ))
            facial_data.append(embedding[i]['embedding'])
    except:
        pass
    return faces,facial_data


def rgb_to_bgr(rgb_color):
    bgr_color = (rgb_color[2], rgb_color[1], rgb_color[0])
    return bgr_color
    
def drawBox(img, x1, y1, x2, y2, l=30, t=5, rt=1, text="Unknown", id=None,display_id=False,draw_rect=False,color=(2, 240, 228),text_color=(255,255,255)):
    # Define the sci-fi style font
    font = cv2.FONT_HERSHEY_SIMPLEX
    fontScale = 0.7
    thickness = 2
    # color = (255, 255, 255)
    color=rgb_to_bgr(color)
    text_color=rgb_to_bgr(text_color)
    # Draw the ID of the detected person on top of the bounding box
    ((id_width, id_height), _) = cv2.getTextSize(str(id), font, fontScale=fontScale, thickness=thickness)
    id_offset_x = x1 + int((x2 - x1 - id_width) / 2)
    id_offset_y = y1 - 35
    if display_id:
        cv2.putText(img, str(id), (id_offset_x, id_offset_y+25), font, fontScale, text_color, thickness)
        # Draw the name of the detected person inside the bounding box
        ((text_width, text_height), _) = cv2.getTextSize(text, font, fontScale=fontScale, thickness=thickness)
        text_offset_x = x1 + int((x2 - x1 - text_width) / 2)
        text_offset_y = y2 + 25
        cv2.putText(img, text, (text_offset_x, text_offset_y), font, fontScale, text_color, thickness)
        # Draw box around face
    if draw_rect:
        cv2.rectangle(img, (x1, y1), (x2, y2), color,thickness=rt)
    t=t-3
    face_width = x2 - x1
    face_height = y2 - y1
    # l = int(l * min(face_width, face_height) / 100)-20
    
    # Draw top-left corner
    cv2.line(img, (x1, y1), (x1 + l, y1), color, thickness=t)
    cv2.line(img, (x1, y1), (x1, y1 + l), color, thickness=t)
    # Draw top-right corner
    cv2.line(img, (x2, y1), (x2 - l, y1), color, thickness=t)
    cv2.line(img, (x2, y1), (x2, y1 + l), color, thickness=t)
    # Draw bottom-left corner
    cv2.line(img, (x1, y2), (x1 + l, y2), color, thickness=t)
    cv2.line(img, (x1, y2), (x1, y2 - l), color, thickness=t)
    # Draw bottom-right corner
    cv2.line(img, (x2, y2), (x2 - l, y2), color, thickness=t)
    cv2.line(img, (x2, y2), (x2, y2 - l), color, thickness=t)
    return img

def white_overlay(img):
    white_img = np.ones_like(img) * 255
    alpha = 0.5
    result = cv2.addWeighted(img, alpha, white_img, 1-alpha, 0)
    x1 = 60
    y1 = 60
    x2 = img.shape[1] - 60
    y2 = img.shape[0] - 60
    mid_x = (img.shape[1]) // 2
    roi = img[y1:y2, x1:x2]
    result[y1:y2, x1:x2] = roi
    return result



def fps_display(img,pTime):
    mid_x = (img.shape[1]) // 2
    fps = 0
    cTime = time.time()
    if cTime - pTime > 0:
        fps = 1 / (cTime - pTime)
    pTime = cTime
    # text = f'FPS: {int(fps)}'
    text=str(int(fps))
    font = cv2.FONT_HERSHEY_PLAIN
    font_scale = 3
    thickness = 3
    text_size = cv2.getTextSize(text, font, font_scale, thickness)[0]
    x = img.shape[1] - text_size[0] - 20
    color=rgb_to_bgr((240, 0, 148))
    cv2.putText(img, text, (mid_x, 45), font, font_scale,color, thickness)
    return img,pTime


   



Directory  /root /.deepface created
Directory  /root /.deepface/weights created


In [4]:
import os
import shutil
if os.path.exists("./download"):
   shutil.rmtree("./download")
if not os.path.exists("./download"):
  os.mkdir("./download")
from bing_image_downloader import downloader
def download_image(query,limit):
  try:
    downloader.download(query, limit=limit, output_dir='./download')
  except:
    pass
celebrity_list=["celebrity","women","men"]
for i in celebrity_list:
  download_image(i,100)    

clear_output()  

In [5]:
import string
import random
def generate_name():
  # Define the length of the file name
  length = 8
  # Define the set of characters to choose from
  chars = string.ascii_letters + string.digits
  # Generate a random file name
  file_name = ''.join(random.choice(chars) for _ in range(length))
  # Print the file name
  return  file_name

  
if os.path.exists("./dataset"):
    shutil.rmtree("./dataset")
#     os.mkdir("./output")
if not os.path.exists("./dataset"):
    os.makedirs("./dataset")

for i in os.listdir("./download"):
  for j in os.listdir(f"./download/{i}"):
    try:
      extenstion=j.split(".")[-1]
      shutil.copy(f"./download/{i}/{j}",f"./dataset/{generate_name()}.{extenstion}")
    except:
      pass    

In [6]:
len(os.listdir("./dataset"))

300

In [7]:
def utils(img):
  # cv2_imshow(img)
  faces,facial_data=deep_data_extract(img)
  if len(faces)!=0 and len(facial_data)!=0:
    if len(faces)==len(facial_data):
        return facial_data[0]
  else:
    return None
        


In [8]:
import csv
def unknown_csv(face_data):
    with open('unknown.csv', mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([face_data, "unknown",0,"None","None"])

In [9]:
import cv2
from google.colab.patches import cv2_imshow
for i in os.listdir("./dataset"):
  img=cv2.imread(f"./dataset/{i}")
  try:
    face_data=utils(img)
    if face_data!=None:
      print(face_data)
      unknown_csv(face_data)
  except:
    pass
clear_output()
from google.colab import files
files.download('/content/unknown.csv')